In [9]:
import pandas as pd
import sklearn.model_selection 
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [10]:
## load the dataset
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
### preprocessin data and dropping irrelevant columns
data = data.drop([['RowNumber','CustomerId','Surname']],axis=1)

KeyError: "[('RowNumber', 'CustomerId', 'Surname')] not found in axis"

In [ ]:
### encode categorical variable
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [12]:
##OHe "geography"
from sklearn.preprocessing import OneHotEncoder
a = OneHotEncoder()
b = a.fit_transform(data[["Geography"]])
b

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [13]:
a.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [14]:
c=pd.DataFrame(b.toarray(),columns=a.get_feature_names_out(["Geography"]))
c

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [15]:
data=pd.concat([data.drop("Geography",axis=1),c],axis=1)
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,619,Female,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,2,15647311,Hill,608,Female,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,3,15619304,Onio,502,Female,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,4,15701354,Boni,699,Female,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,5,15737888,Mitchell,850,Female,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('b.pkl','wb') as file:
    pickle.dump(b,file)

In [69]:
##Divide dataset into independent and dependent features
from sklearn.model_selection import train_test_split
X = data.drop("Exited", axis=1)
y = data["Exited"]
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

##Scaling Features
scaler = StandardScaler()
X_train =scaler.fit_transform(X_train)
X_test =scaler.transform(X_test)

ValueError: could not convert string to float: "P'an"

In [ ]:
with open("scaler.pkl",'wb') as file:
    pickle.dump(scaler,file
                
                )

In [ ]:
def generate_sequence(n):
    return ''.join(str(i) for i in range(1, n+1))

print(generate_sequence(5))

In [ ]:
def generate_sequence(n):
    return ''.join(str(i) for i in range(1, n+1))

print(generate_sequence(5))

12345


In [33]:
####  ANN implementstion
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [28]:
### build our ann model
model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(32, activation="relu"),  # HL2
    # output layer
])


c:\code\Python\venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
model.summary()
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,104 (12.12 KB)

 Trainable params: 3,104 (12.12 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
loss=tf.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x0000024424426160>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [40]:
###Compile model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=["accuracy"])

In [56]:
### stet up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="logs/fit/"+datetime.datetime.now().strftime("%d/%m/%Y, %H:%M:%S")
tensorflow_callback=TensorBoard()

In [44]:
###Set up EarlyStopping
early_stopping_callback=EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True)

In [57]:
###Training the model
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

ValueError: Invalid dtype: object

In [50]:
model.save("model.h5")

In [51]:
%load_ext tensorboard

In [55]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 15868), started 0:03:33 ago. (Use '!kill 15868' to kill it.)

In [ ]:
from tensorflow.keras.models import load_model

### Losd the ann model,scaler,pickle,onehot
model= load_model("model.h5")

## load the encoder and scaler
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)
with open('b.pkl','rb') as file:
    b = pickle.load(file)
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)